In [24]:
import ray
import os
from pyhdfs import HdfsClient
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost_ray import RayXGBRFRegressor
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold

In [2]:
nodes = ["127.0.0.1"]
client = HdfsClient(hosts=nodes, user_name="bigdata2022-VirtualBox")
df=pd.read_csv(client.open("hdfs://localhost:9870/user/bigdata2022/datasets/big_data_project/taxi_dataset_EDA.csv"))

In [3]:
df.head()
seed=42

In [4]:
X = df.drop(['trip_duration'], axis=1)
t = df['trip_duration']

In [5]:
X_train, X_test, t_train, t_test = train_test_split(X, t, train_size=0.75, random_state=seed)

In [ ]:
start_time=time.time()

In [6]:
model = RayXGBRFRegressor(n_jobs=4, random_state=seed)

In [9]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
param_grid = {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
              'min_child_weight': np.arange(0.0001, 0.5, 0.001),
              'gamma': np.arange(0.0,40.0,0.005),
              'learning_rate': np.arange(0.0005,0.3,0.0005),
              'subsample': np.arange(0.01,1.0,0.01),
              'colsample_bylevel': np.round(np.arange(0.1,1.0,0.01)),
              'colsample_bytree': np.arange(0.1,1.0,0.01)}

In [26]:
kfold = KFold(n_splits=4, shuffle=True, random_state=10)
grid_search = GridSearchCV(model, param_grid, scoring="accuracy", cv=5)

In [ ]:
grid_result = grid_search.fit(X_train,t_train)

2022-11-21 22:31:40,026	INFO worker.py:1519 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [14]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
print(time.time()-start_time)